In [47]:
import pandas as pd
import seaborn as sns
import re
from pandasql import sqldf
import statistics
from tqdm import tqdm
pysqldf = lambda q: sqldf(q, globals())


In [2]:
prop_list = pd.read_csv("../data/raw/listing_properties.csv")

In [3]:
prop_list

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
0,0,1,1,0,$330,$330,"409/757 Bourke, Docklands VIC 3008",basic,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,1433.0,14303265,NaN,NaN,NaN,NaN
1,1,2,1,1,$430 pw,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",branded,rent,Apartment / Unit / Flat,new,rent,False,0,1868.0,16089992,"Monday, 28th June 2021",NaN,NaN,NaN
2,2,4,3,2,$585,$585,"1 Liverpool Circuit, Craigieburn VIC 3064",branded,rent,House,recentlyUpdated,rent,False,0,NaN,15279569,"Friday, 25th November 2022",NaN,NaN,NaN
3,3,3,1,1,$420,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",branded,rent,House,new,rent,False,0,1820.0,16089977,"Monday, 26th September 2022",NaN,NaN,NaN
4,4,1,1,1,$640 per week,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,2781.0,16089972,"Monday, 22nd August 2022",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13748,13748,4,2,2,$400 Per Week pw,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",basic,rent,House,live,rent,False,0,1738.0,15434092,"Wednesday, 27th October 2021",NaN,NaN,NaN
13749,13749,1,1,0,$435 per week,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",basic,rent,Apartment / Unit / Flat,live,rent,False,0,2500.0,14409092,"Saturday, 9th October 2021",NaN,50.0,NaN
13750,13750,3,2,4,$350 Per Week pw,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",basic,rent,House,live,rent,False,0,1521.0,15430756,"Wednesday, 6th October 2021",NaN,NaN,NaN
13751,13751,4,2,1,$600,$600,"18 Jessie Street, Preston VIC 3072",basic,rent,House,live,rent,False,0,2600.0,15428527,"Tuesday, 5th October 2021",NaN,NaN,NaN


In [4]:
prop_list[prop_list['landArea'].notna()]

,Unnamed: 0,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,bond,id,availableFrom,landArea,buildingArea,method
11,11,4,2,2,$630 weekly,$630 weekly,"90 Shepherd Rd, Glen Waverley VIC 3150",basic,rent,House,new,rent,False,0,2738.0,16089938,"Friday, 23rd September 2022",652.0,175.0,NaN
13,13,2,1,0,$700 per week,$700 per week,"15 Dally Street, Clifton Hill VIC 3068",basic,rent,House,recentlyUpdated,rent,False,0,3042.0,15478015,"Friday, 14th October 2022",158.0,95.0,NaN
14,14,4,2,2,$500,$500,"20 Knightsford Avenue, Clyde VIC 3978",branded,rent,House,new,rent,False,0,2173.0,16089930,"Wednesday, 12th October 2022",375.0,NaN,NaN
18,18,3,1,2,$360 Per Week,$360 Per Week,"5 Roxburgh Road, Yallourn North VIC 3825",branded,rent,House,new,rent,False,0,1560.0,16089911,"Wednesday, 5th October 2022",634.0,NaN,NaN
35,35,3,2,2,$460 pw,$460 pw,"2/17 Nelson Road, Trafalgar VIC 3824",basic,rent,Townhouse,new,rent,False,0,1999.0,16089810,"Sunday, 25th September 2022",317.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13667,13667,3,2,11,$1180 PER WEEK $5113 PER MONTH,$1180 PER WEEK $5113 PER MONTH,"2531 STEIGLITZ ROAD, Steiglitz VIC 3331",branded,rent,Acreage / Semi-Rural,live,rent,True,0,5000.0,15714546,"Sunday, 20th February 2022",600000.0,NaN,NaN
13670,13670,4,2,2,Contact Agent,Contact Agent,"13 Roaming Drive, Fraser Rise VIC 3336",branded,rent,House,live,rent,False,0,NaN,15711847,"Wednesday, 6th April 2022",352.0,NaN,NaN
13677,13677,2,2,1,UNDER APPLICATION,UNDER APPLICATION,"2/50 Fraser Crescent, Ocean Grove VIC 3226",branded,rent,Townhouse,live,rent,False,0,2086.0,15649364,"Saturday, 5th February 2022",197.0,NaN,NaN
13732,13732,4,2,2,$410.00 Per Week pw,$410.00 Per Week pw,"10 Clearwood Drive, Truganina VIC 3029",basic,rent,House,live,rent,False,0,1782.0,15454576,"Friday, 10th December 2021",360.0,22.0,NaN


In [4]:
prop_subset = prop_list[['id','price','beds','baths','parking','title','address','propertyType','status','bond']]

In [5]:
prop_subset.isnull().sum()

id                0
price             0
beds              0
baths             0
parking           0
title             0
address           0
propertyType      0
status            0
bond            890
dtype: int64

In [6]:
prop_subset_clean = prop_subset.dropna()

In [7]:
prop_subset_clean.isnull().sum()

id              0
price           0
beds            0
baths           0
parking         0
title           0
address         0
propertyType    0
status          0
bond            0
dtype: int64

In [8]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,$330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,$430 pw,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,$420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,$640 per week,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,$550 PER WEEK,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,$400 Per Week pw,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,$435 per week,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,$350 Per Week pw,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,$600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [10]:
def get_price(e):
    s = re.search('\$[0-9]*\,?\d*\.?\d*', e)
    if (s == None):
        return None
    else:
        final = s.group().strip('$')
        final = final.replace(',','')
        return final

In [11]:
prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)

/tmp/ipykernel_392/2730741709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = prop_subset_clean['price'].apply(get_price)


In [12]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [13]:
prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])

/tmp/ipykernel_392/2753503525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_subset_clean['price'] = pd.to_numeric(prop_subset_clean['price'])


In [14]:
prop_subset_clean

,id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
3,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
4,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
6,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
13748,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
13749,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
13750,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
13751,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [15]:
prop_subset_clean = prop_subset_clean.dropna(axis=0)

In [21]:
prop_subset_clean = prop_subset_clean.reset_index()

In [22]:
prop_subset_clean['price'].sort_values(ascending = True)

7862        40.0
10950       40.0
12075       50.0
4277        50.0
12350       50.0
          ...   
6364      4000.0
11403     5000.0
7512      5500.0
10550    39000.0
11644    41250.0
Name: price, Length: 12487, dtype: float64

In [23]:
prop_subset_clean.iloc[11644,:]

index                                                       12751
id                                                       15994278
price                                                     41250.0
beds                                                            0
baths                                                           0
parking                                                         0
title                                                   $41250 pa
address         Level 1, 301/13-15 Lake Street, Caroline Sprin...
propertyType                              Apartment / Unit / Flat
status                                                       live
bond                                                       9519.0
Name: 11644, dtype: object

In [26]:
prop_subset_clean = prop_subset_clean[prop_subset_clean['beds'] > 0]

In [29]:
prop_subset_clean.pop('index')

0            0
1            1
2            3
3            4
4            6
         ...  
12482    13748
12483    13749
12484    13750
12485    13751
12486    13752
Name: index, Length: 12405, dtype: int64

In [32]:
prop_subset_clean.to_csv('../data/curated/cleaned_property_listing_data.csv')

### Now we clean all existing scraped dataset

In [13]:
prop = pd.read_csv('../data/curated/cleaned_property_listing_data.csv')
schools = pd.read_csv('../data/raw/closest_school.csv')
suburb_stat = pd.read_csv('../data/raw/suburb_stat.csv')
neigh_stat = pd.read_csv('../data/raw/neigh_stat.csv')

In [28]:
prop['bed'] = pd.to_numeric(prop['beds'])
prop['baths'] = pd.to_numeric(prop['baths'])
prop['parking'] = pd.to_numeric(prop['parking'])
prop['bond'] = pd.to_numeric(prop['bond'])

In [30]:
prop.pop('bed')

0        1
1        2
2        3
3        1
4        4
        ..
12400    4
12401    1
12402    3
12403    4
12404    2
Name: bed, Length: 12405, dtype: int64

In [32]:
prop.to_csv('../data/curated/cleaned_property_listing_data.csv')

In [19]:
neigh_stat

,Unnamed: 0,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0,0.129199,0.642119,0.195090,0.033592,0.136929,0.295322,0.704678,0.512945,0.487055,14303265
1,1,0.210918,0.248139,0.332506,0.208437,0.478632,0.639535,0.360465,0.576052,0.423948,16089992
2,2,0.328173,0.229102,0.346749,0.095975,0.000000,0.800000,0.200000,0.647887,0.352113,15279569
3,3,0.331512,0.251366,0.320583,0.096539,0.604072,0.777778,0.222222,0.600000,0.400000,16089977
4,4,0.093800,0.356121,0.356121,0.193959,0.202410,0.537366,0.462633,0.568019,0.431981,16089972
...,...,...,...,...,...,...,...,...,...,...,...
13711,13711,0.362153,0.427406,0.205546,0.004894,0.000000,0.374233,0.625767,0.602015,0.397985,15434092
13712,13712,0.113907,0.540397,0.255629,0.090066,0.240545,0.355030,0.644970,0.419152,0.580848,14409092
13713,13713,0.233898,0.277966,0.315254,0.172881,0.642857,0.663366,0.336634,0.570732,0.429268,15430756
13714,13714,0.208772,0.356140,0.231579,0.203509,0.662698,0.641304,0.358696,0.493088,0.506912,15428527


In [20]:
schools.pop("Unnamed: 0")
suburb_stat.pop("Unnamed: 0")
prop.pop('Unnamed: 0')
neigh_stat.pop('Unnamed: 0')

0            0
1            1
2            2
3            3
4            4
         ...  
13711    13711
13712    13712
13713    13713
13714    13714
13715    13715
Name: Unnamed: 0, Length: 13716, dtype: int64

In [21]:
neigh_stat

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0.129199,0.642119,0.195090,0.033592,0.136929,0.295322,0.704678,0.512945,0.487055,14303265
1,0.210918,0.248139,0.332506,0.208437,0.478632,0.639535,0.360465,0.576052,0.423948,16089992
2,0.328173,0.229102,0.346749,0.095975,0.000000,0.800000,0.200000,0.647887,0.352113,15279569
3,0.331512,0.251366,0.320583,0.096539,0.604072,0.777778,0.222222,0.600000,0.400000,16089977
4,0.093800,0.356121,0.356121,0.193959,0.202410,0.537366,0.462633,0.568019,0.431981,16089972
...,...,...,...,...,...,...,...,...,...,...
13711,0.362153,0.427406,0.205546,0.004894,0.000000,0.374233,0.625767,0.602015,0.397985,15434092
13712,0.113907,0.540397,0.255629,0.090066,0.240545,0.355030,0.644970,0.419152,0.580848,14409092
13713,0.233898,0.277966,0.315254,0.172881,0.642857,0.663366,0.336634,0.570732,0.429268,15430756
13714,0.208772,0.356140,0.231579,0.203509,0.662698,0.641304,0.358696,0.493088,0.506912,15428527


In [22]:
prop.rename(columns={'id':'listing_id'}, inplace = True)
suburb_stat.rename(columns={'id':'listing_id'}, inplace = True)
neigh_stat.rename(columns={'id':'listing_id'}, inplace = True)

In [36]:
neigh_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13716 entries, 0 to 13715
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age0To19          13716 non-null  float64
 1   age20To39         13716 non-null  float64
 2   age40To59         13716 non-null  float64
 3   age60Plus         13716 non-null  float64
 4   longTermResident  13716 non-null  float64
 5   owner             13716 non-null  float64
 6   renter            13716 non-null  float64
 7   family            13716 non-null  float64
 8   single            13716 non-null  float64
 9   listing_id        13716 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 1.0 MB


In [37]:
suburb_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13716 entries, 0 to 13715
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   suburb           13716 non-null  object 
 1   medianRentPrice  13716 non-null  int64  
 2   population       13716 non-null  int64  
 3   avgAge           12892 non-null  object 
 4   owners           13716 non-null  float64
 5   renters          13716 non-null  float64
 6   families         13716 non-null  float64
 7   singles          13716 non-null  float64
 8   listing_id       13716 non-null  int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 964.5+ KB


In [24]:
schools.rename(columns={'name':"school_name", 'address':'school_address'})

,id,educationLevel,school_name,distance,state,postCode,year,gender,type,school_address,url,listing_id
0,NaN,combined,Eltham College - King Street Campus,708.921822,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
1,NaN,secondary,Hester Hornbrook Academy - City Campus,802.849893,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
2,NaN,secondary,Ozford College - Ozford College Campus,899.683164,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,14303265
3,NaN,secondary,Ozford College,900.734191,VIC,3000,11-12,CoEd,Private,"Melbourne, VIC 3000",http://www.ozford.edu.au,14303265
4,NaN,primary,Docklands Primary School,916.135369,VIC,3008,Prep-6,CoEd,Government,"Docklands, VIC 3008",NaN,14303265
...,...,...,...,...,...,...,...,...,...,...,...,...
167001,1999.0,secondary,Simonds Catholic College,942.720484,VIC,3003,7-12,Boys,Catholic,"West Melbourne, VIC 3003",http://www.sccmelb.catholic.edu.au,15384018
167002,NaN,secondary,St Joseph's Flexible Learning Centre,969.075211,VIC,3051,7-12,CoEd,Catholic,"North Melbourne, VIC 3051",http://www.youthplus.edu.au,15384018
167003,NaN,combined,Eltham College - King Street Campus,1012.666859,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN,15384018
167004,1266.0,primary,Carlton Gardens Primary School,1058.486337,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au,15384018


In [43]:
prop

,listing_id,price,beds,baths,parking,title,address,propertyType,status,bond
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0
2,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0
3,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0
4,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0
...,...,...,...,...,...,...,...,...,...,...
12400,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0
12401,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0
12402,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0
12403,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0


In [58]:
prop['address']
suburbs = []
postcode = []
for a in prop['address']:
    temp = a.split("VIC")
    sub = temp[0].split(",")[-1]
#     print(sub)
    suburbs.append(sub.strip())
    postcode.append(int(temp[1]))
    
    
prop['suburb'] = suburbs
prop['postcode'] = postcode

In [60]:
prop.to_csv('../data/curated/cleaned_property_listing_data.csv')

### Getting Aggregated data by type of rental property and suburb

In [1]:
%cd ../scripts
from get_agg import get_agg
agg = get_agg(write=True) # sdf is saved to a location
%cd ../notebooks

/mnt/c/Users/User/OneDrive/Desktop/Uni - 2022/sem2/Applied Data Science/Proj2/generic-real-estate-consulting-project-group-25/scripts


100%|███████████████████████████████████████████████████████████████████████████████| 2935/2935 [04:50<00:00, 10.10it/s]

/mnt/c/Users/User/OneDrive/Desktop/Uni - 2022/sem2/Applied Data Science/Proj2/generic-real-estate-consulting-project-group-25/notebooks


In [2]:
agg

,beds,avg_price,count,propertyType,suburb,median
0,1,500.0,1,Acreage / Semi-Rural,Oaklands Junction,500.0
1,1,401.0,15,Apartment / Unit / Flat,Abbotsford,410.0
2,1,290.0,1,Apartment / Unit / Flat,Airport West,290.0
3,1,417.5,2,Apartment / Unit / Flat,Albert Park,417.5
4,1,320.0,2,Apartment / Unit / Flat,Alphington,320.0
...,...,...,...,...,...,...
2930,7,1250.0,1,House,Brunswick,1250.0
2931,7,170.0,1,House,Dandenong,170.0
2932,8,1950.0,1,House,Balaclava,1950.0
2933,8,1300.0,1,House,Mordialloc,1300.0


### Checking Historical Data

In [78]:
bed_1_flat = pd.read_csv('../data/raw/Rental_Price_historical/1_bed_flat.csv')

In [79]:
bed_1_flat

,Unnamed: 0,Unnamed: 1,Mar-00,Unnamed: 3,Jun-00,Unnamed: 5,Sep-00,Unnamed: 7,Dec-00,Unnamed: 9,...,Mar-20,Unnamed: 163,Jun-20,Unnamed: 165,Sep-20,Unnamed: 167,Dec-20,Unnamed: 169,Mar-21,Unnamed: 171
0,NaN,NaN,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
1,Inner Melbourne,Albert Park-Middle Park-West St Kilda,352,$165,346,$165,376,$170,367,$175,...,230,$370,228,$355,231,$350,243,$350,263,$340
2,NaN,Armadale,210,$150,210,$150,211,$155,211,$160,...,205,$375,208,$368,170,$350,184,$350,206,$350
3,NaN,Carlton North,86,$150,77,$155,74,$150,65,$150,...,70,$363,67,$360,72,$350,64,$350,68,$338
4,NaN,Carlton-Parkville,297,$170,297,$170,310,$175,343,$180,...,"3,354",$360,"3,239",$356,"2,919",$350,"2,829",$340,"2,097",$300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,NaN,Wanagaratta,51,$85,45,$85,43,$85,46,$85,...,69,$195,67,$195,61,$190,56,$195,52,$195
156,NaN,Warragul,13,$80,11,$75,12,$90,10,$90,...,-,-,-,-,-,-,-,-,-,-
157,NaN,Warrnambool,113,$75,103,$75,107,$75,103,$80,...,61,$200,65,$220,63,$220,63,$230,55,$250
158,NaN,Wodonga,76,$85,72,$85,77,$85,83,$85,...,46,$180,57,$185,56,$185,56,$185,59,$200


### Joining

In [25]:
joined = pd.merge(prop, suburb_stat, how='left', on='listing_id')
# joined_with_school = pd.merge(prop, schools, how='right', on='listing_id')

In [26]:
joined

,listing_id,price,beds,baths,parking,title,address,propertyType,status,bond,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles
0,14303265,330.0,1,1,0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,Docklands,380.0,10962.0,20 to 39,0.356243,0.643757,0.393689,0.606311
1,16089992,430.0,2,1,1,$430 pw,"38/60-68 Gladesville Boulevard, Patterson Lake...",Apartment / Unit / Flat,new,1868.0,Patterson Lakes,430.0,7565.0,40 to 59,0.835075,0.164925,0.527570,0.472430
2,16089977,420.0,3,1,1,$420,"8 Stoneman Turn, Caroline Springs VIC 3023",House,new,1820.0,Caroline Springs,410.0,24200.0,20 to 39,0.766538,0.233462,0.571566,0.428434
3,16089972,640.0,1,1,1,$640 per week,"1510/1 Freshwater Place, Southbank VIC 3006",Apartment / Unit / Flat,new,2781.0,Southbank,360.0,18706.0,20 to 39,0.347128,0.652872,0.312419,0.687581
4,16089969,550.0,4,2,2,$550 PER WEEK,"6 Plenty Drive, Warragul VIC 3820",House,new,2390.0,Warragul,450.0,15757.0,60+,0.757297,0.242703,0.505706,0.494294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12432,15434092,400.0,4,2,2,$400 Per Week pw,"7 Ebony Way, Tarneit VIC 3029",House,live,1738.0,Tarneit,400.0,34556.0,20 to 39,0.699615,0.300385,0.611806,0.388194
12433,14409092,435.0,1,1,0,$435 per week,"17/416 St Kilda Rd, Melbourne VIC 3004",Apartment / Unit / Flat,live,2500.0,Melbourne,375.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000
12434,15430756,350.0,3,2,4,$350 Per Week pw,"16 Goulburn Court, Werribee VIC 3030",House,live,1521.0,Werribee,340.0,40348.0,20 to 39,0.679556,0.320444,0.463697,0.536303
12435,15428527,600.0,4,2,1,$600,"18 Jessie Street, Preston VIC 3072",House,live,2600.0,Preston,600.0,32856.0,20 to 39,0.588015,0.411985,0.399137,0.600863


In [9]:
joined_with_school

,listing_id,price,beds,baths,parking,title,address_x,propertyType,status,bond,...,educationLevel,name,distance,state,postCode,year,gender,type,address_y,url
0,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,combined,Eltham College - King Street Campus,708.921822,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
1,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Hester Hornbrook Academy - City Campus,802.849893,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
2,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Ozford College - Ozford College Campus,899.683164,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
3,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,secondary,Ozford College,900.734191,VIC,3000,11-12,CoEd,Private,"Melbourne, VIC 3000",http://www.ozford.edu.au
4,14303265,330.0,1.0,1.0,0.0,$330,"409/757 Bourke, Docklands VIC 3008",Apartment / Unit / Flat,recentlyUpdated,1433.0,...,primary,Docklands Primary School,916.135369,VIC,3008,Prep-6,CoEd,Government,"Docklands, VIC 3008",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167377,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,secondary,Simonds Catholic College,942.720484,VIC,3003,7-12,Boys,Catholic,"West Melbourne, VIC 3003",http://www.sccmelb.catholic.edu.au
167378,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,secondary,St Joseph's Flexible Learning Centre,969.075211,VIC,3051,7-12,CoEd,Catholic,"North Melbourne, VIC 3051",http://www.youthplus.edu.au
167379,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,combined,Eltham College - King Street Campus,1012.666859,VIC,3000,NaN,NaN,Private,"Melbourne, VIC 3000",NaN
167380,15384018,750.0,2.0,1.0,0.0,$750/W Furnished,"2509/228 La Trobe Street, Melbourne VIC 3000",Apartment / Unit / Flat,live,3258.0,...,primary,Carlton Gardens Primary School,1058.486337,VIC,3053,Prep-6,CoEd,Government,"Carlton, VIC 3053",http://www.carltongardens.vic.edu.au


In [10]:
joined_with_school.to_csv('../data/curated/joined_with_school.csv')